<a href="https://colab.research.google.com/github/davidtkeane/Google_Colab/blob/main/mistral_finetune_7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting starting fine-tuning Mistral 7B

This notebook shows you a simple example of how to LoRA finetune Mistral 7B. You can can run this notebook in Google Colab with Pro + account with A100 and 40GB RAM.

<a target="_blank" href="https://colab.research.google.com/github/mistralai/mistral-finetune/blob/main/tutorials/mistral_finetune_7b.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


Check out `mistral-finetune` Github repo to learn more: https://github.com/mistralai/mistral-finetune/

## Installation

Clone the `mistral-finetune` repo:


In [ ]:
!pip install colorama

This will take about 45 to an hour of your life wasted watching this happen. I hope you are proud of yourself.

In [ ]:
# prompt: welcome banner

import sys

def print_welcome_banner():
  """Prints a colorful welcome banner to the console."""

  banner = r"""
  ____  _          _   _      _ _         _     _
 |  _ \| | ___   _| |_| | ___| | |___ ___| | __| | ___ _ __ ___
 | | | | |/ _ \ / _`| |/ _ \ | |/ _ \/ __| |/ _` |/ _ \ '__/ __|
 | |_| | |  __/ (_| | |  __/ | |  __/ (__| | (_| |  __/ |  \__ \
 |____/|_|\___|\__,_|_|\___|_|_|\___|\___|_|\__,_|\___|_|  |___/

 """

  print("\033[92m" + banner + "\033[0m") # Green color
  print("\033[94mWelcome to the Mistral 7B Fine-tuning Notebook!\033[0m") # Blue color
  print("\033[93mThis notebook demonstrates a simple example of how to LoRA fine-tune Mistral 7B.\033[0m") # Yellow
  print("Please be patient while the installation completes.\n") # Reset color


if __name__ == "__main__":
  print_welcome_banner()

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)

if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print(f"Your runtime has {ram_gb:.1f} gigabytes of available RAM")

!nvidia-smi

In [ ]:
#!/usr/bin/python3

# Loads Google Drive

import os
import shutil
from google.colab import drive
from colorama import Fore, Style, init
from IPython.display import display, HTML

# Initialize colorama
init()

# Access the API key from Colab Secrets
from google.colab import userdata

try:
    api_key = userdata.get('GOOGLE_DRIVE_API')
    print(f"{Fore.GREEN}✅ API Key loaded successfully.{Style.RESET_ALL}")

    # Set the API key as an environment variable
    os.environ['GOOGLE_DRIVE_API'] = api_key
except Exception as e:
    print(f"{Fore.RED}❌ Error loading API Key: {e}{Style.RESET_ALL}")
    api_key = None

# Mount Google Drive using the API key (if available)
if api_key:
    try:
        # Use the API key to authenticate and mount Google Drive
        drive.mount('/content/drive', force_remount=True)
        print(f"{Fore.GREEN}✅ Google Drive mounted successfully using API Key.{Style.RESET_ALL}")
    except Exception as e:
        print(f"{Fore.RED}❌ Error mounting Google Drive: {e}{Style.RESET_ALL}")
else:
    print(f"{Fore.YELLOW}⚠️ No API Key found. Please log in manually.{Style.RESET_ALL}")
    drive.mount('/content/drive', force_remount=True)

# Define the Google Drive folder path
drive_folder = "/content/drive/MyDrive/Colab_Projects"

# Define the Colab working directory
colab_folder = "/content/"

# Ensure the Colab folder exists
os.makedirs(colab_folder, exist_ok=True)

# Function to count files in a directory
def count_files(directory):
    return sum([len(files) for _, _, files in os.walk(directory)])

# Function to sync files from Colab to Google Drive
def sync_to_drive():
    try:
        print(f"{Fore.CYAN}🔄 Syncing files from Colab to Google Drive...{Style.RESET_ALL}")

        # Use rsync to copy files from Colab to Google Drive
        os.system(f"rsync -av --progress {colab_folder}/ {drive_folder}/")

        print(f"{Fore.GREEN}✅ Files synced from Colab to Google Drive.{Style.RESET_ALL}")
    except Exception as e:
        print(f"{Fore.RED}❌ Error syncing to Google Drive: {e}{Style.RESET_ALL}")

# Function to sync files from Google Drive to Colab
def sync_from_drive():
    try:
        print(f"{Fore.CYAN}🔄 Syncing files from Google Drive to Colab...{Style.RESET_ALL}")

        # Count files before sync
        initial_count = count_files(colab_folder)

        # Use rsync to copy files from Google Drive to Colab
        # -a: Archive mode (recursive, preserves permissions, symlinks, etc.)
        # -v: Verbose output
        # --progress: Show progress during transfer
        # --ignore-existing: Skip files that already exist in the destination
        # old command = os.system(f"rsync -av --progress {drive_folder}/ {colab_folder}/")
        os.system(f"rsync -av --progress {drive_folder} {colab_folder}/")
        # Count files after sync
        final_count = count_files(colab_folder)
        files_transferred = final_count - initial_count

        print(f"{Fore.GREEN}✅ Sync completed!{Style.RESET_ALL}")
        print(f"{Fore.GREEN}📂 Files transferred: {files_transferred}{Style.RESET_ALL}")

        # List files in the Colab folder to confirm sync
        print(f"{Fore.BLUE}📁 Files in Colab folder:{Style.RESET_ALL}")
        for root, dirs, files in os.walk(colab_folder):
            for file in files:
                print(os.path.join(root, file))
    except Exception as e:
        print(f"{Fore.RED}❌ Error syncing from Google Drive: {e}{Style.RESET_ALL}")

# Function to display a sync button
def display_sync_button():
    display(HTML('''
        <div>
            <button onclick="syncFromDrive()">Sync from Google Drive to Colab</button>
            <button onclick="syncToDrive()">Sync from Colab to Google Drive</button>
            <button onclick="refreshPage()">Refresh</button>
        </div>
        <script>
            function syncFromDrive() {
                google.colab.kernel.invokeFunction('sync_from_drive', [], {});
            }
            function syncToDrive() {
                google.colab.kernel.invokeFunction('sync_to_drive', [], {});
            }
            function refreshPage() {
                window.location.reload();
            }
        </script>
    '''))

# Register the sync functions with Colab
from google.colab import output
output.register_callback('sync_from_drive', sync_from_drive)
output.register_callback('sync_to_drive', sync_to_drive)


# Display the sync button
display_sync_button()

✅ API Key loaded successfully.
Mounted at /content/drive
✅ Google Drive mounted successfully using API Key.


In [ ]:
%cd /content/drive/MyDrive/Colab_Projects/
!git clone https://github.com/mistralai/mistral-finetune.git

/content/drive/MyDrive/Colab_Projects
Cloning into 'mistral-finetune'...
remote: Enumerating objects: 472, done.
remote: Counting objects: 100% (249/249), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 472 (delta 211), reused 159 (delta 159), pack-reused 223 (from 2)
Receiving objects: 100% (472/472), 243.32 KiB | 2.36 MiB/s, done.
Resolving deltas: 100% (251/251), done.


Install all required dependencies:

In [ ]:
!pip install -r /content/drive/MyDrive/Colab_Projects/mistral-finetune/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.2/218.2 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 16.5 MB/s eta 0:00:0

## Model download

In [ ]:
!wget https://models.mistralcdn.com/mistral-7b-v0-3/mistral-7B-v0.3.tar

--2024-05-24 18:50:25--  https://models.mistralcdn.com/mistral-7b-v0-3/mistral-7B-v0.3.tar
Resolving models.mistralcdn.com (models.mistralcdn.com)... 104.26.6.117, 104.26.7.117, 172.67.70.68, ...
Connecting to models.mistralcdn.com (models.mistralcdn.com)|104.26.6.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14496675840 (14G) [application/x-tar]
Saving to: ‘mistral-7B-v0.3.tar’

mistral-7B-v0.3.tar 100%[===================>]  13.50G  40.5MB/s    in 6m 3s   

2024-05-24 18:56:29 (38.1 MB/s) - ‘mistral-7B-v0.3.tar’ saved [14496675840/14496675840]



In [ ]:
!wget -P /content/drive/MyDrive/Colab_Projects/mistral-finetune/mistral_models/ https://models.mistralcdn.com/mistral-7b-v0-3/mistral-7B-v0.3.tar

--2025-01-08 23:42:16--  https://models.mistralcdn.com/mistral-7b-v0-3/mistral-7B-v0.3.tar
Resolving models.mistralcdn.com (models.mistralcdn.com)... 104.26.6.117, 104.26.7.117, 172.67.70.68, ...
Connecting to models.mistralcdn.com (models.mistralcdn.com)|104.26.6.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14496675840 (14G) [application/x-tar]
Saving to: ‘/content/drive/MyDrive/Colab_Projects/mistral-finetune/mistral_models/mistral-7B-v0.3.tar’

mistral-7B-v0.3.tar 100%[===================>]  13.50G  25.6MB/s    in 10m 54s 

2025-01-08 23:53:11 (21.1 MB/s) - ‘/content/drive/MyDrive/Colab_Projects/mistral-finetune/mistral_models/mistral-7B-v0.3.tar’ saved [14496675840/14496675840]



In [ ]:
!DIR=/content/drive/MyDrive/Colab_Projects/mistral-finetune/mistral_models && tar -xf /content/drive/MyDrive/Colab_Projects/mistral-finetune/mistral_models/mistral-7B-v0.3.tar -C $DIR

In [ ]:
# Alternatively, you can download the model from Hugging Face

# !pip install huggingface_hub
# from huggingface_hub import snapshot_download
# from pathlib import Path

# mistral_models_path = Path.home().joinpath('mistral_models', '7B-v0.3')
# mistral_models_path.mkdir(parents=True, exist_ok=True)

# snapshot_download(repo_id="mistralai/Mistral-7B-v0.3", allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"], local_dir=mistral_models_path)

#! cp -r /root/mistral_models/7B-v0.3 /content/mistral_models
#! rm -r /root/mistral_models/7B-v0.3

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer.model.v3:   0%|          | 0.00/587k [00:00<?, ?B/s]

params.json:   0%|          | 0.00/202 [00:00<?, ?B/s]

consolidated.safetensors:   0%|          | 0.00/14.5G [00:00<?, ?B/s]

'/root/mistral_models/7B-v0.3'

In [ ]:
!ls /content/drive/MyDrive/Colab_Projects/mistral-finetune/mistral_models/

consolidated.safetensors  mistral-7B-v0.3.tar  params.json  tokenizer.model.v3


## Prepare dataset

To ensure effective training, mistral-finetune has strict requirements for how the training data has to be formatted. Check out the required data formatting [here](https://github.com/mistralai/mistral-finetune/tree/main?tab=readme-ov-file#prepare-dataset).

In this example, let’s use the ultrachat_200k dataset. We load a chunk of the data into Pandas Dataframes, split the data into training and validation, and save the data into the required `jsonl` format for fine-tuning.

In [ ]:
%cd /content/drive/MyDrive/Colab_Projects/mistral-finetune/

/content/drive/MyDrive/Colab_Projects/mistral-finetune


In [ ]:
# make a new directory called data
!mkdir -p data

In [ ]:
# navigate to this data directory
%cd /content/drive/MyDrive/Colab_Projects/mistral-finetune/data

/content/drive/MyDrive/Colab_Projects/mistral-finetune/data


In [ ]:
# read data into a pandas dataframe
import pandas as pd

df = pd.read_parquet('https://huggingface.co/datasets/HuggingFaceH4/ultrachat_200k/resolve/main/data/test_gen-00000-of-00001-3d4cd8309148a71f.parquet')

In [ ]:
# split data into training and evaluation
df_train=df.sample(frac=0.95,random_state=200)
df_eval=df.drop(df_train.index)

In [ ]:
# save data into .jsonl files
df_train.to_json("ultrachat_chunk_train.jsonl", orient="records", lines=True)
df_eval.to_json("ultrachat_chunk_eval.jsonl", orient="records", lines=True)

In [ ]:
!ls /content/drive/MyDrive/Colab_Projects/mistral-finetune/data

ultrachat_chunk_eval.jsonl  ultrachat_chunk_train.jsonl


In [ ]:
# navigate to the mistral-finetune directory
%cd /content/drive/MyDrive/Colab_Projects/mistral-finetune/

/content/drive/MyDrive/Colab_Projects/mistral-finetune


In [ ]:
# some of the training data doesn't have the right format,
# so we need to reformat the data into the correct format and skip the cases that doesn't have the right format:

!python -m utils.reformat_data /content/drive/MyDrive/Colab_Projects/mistral-finetune/data/ultrachat_chunk_train.jsonl

Skipped 3674th sample
Skipped 9176th sample
Skipped 10559th sample
Skipped 13293th sample
Skipped 13973th sample
Skipped 15219th sample


In [ ]:
# eval data looks all good
!python -m utils.reformat_data /content/drive/MyDrive/Colab_Projects/mistral-finetune/data/ultrachat_chunk_eval.jsonl

In [ ]:
# Now you can verify your training yaml to make sure the data is correctly formatted and to get an estimate of your training time.

!python -m utils.validate_data --train_yaml /content/drive/MyDrive/Colab_Projects/mistral-finetune/example/7B.yaml


Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/content/drive/MyDrive/Colab_Projects/mistral-finetune/utils/validate_data.py", line 372, in <module>
    main(args)
  File "/content/drive/MyDrive/Colab_Projects/mistral-finetune/utils/validate_data.py", line 179, in main
    datasets, weights = parse_data_sources(pretrain_file, instruct_file)
  File "/content/drive/MyDrive/Colab_Projects/mistral-finetune/finetune/data/dataset.py", line 159, in parse_data_sources
    assert min(n_weights) > 0
ValueError: min() arg is an empty sequence


## Start training

In [ ]:
# these info is needed for training
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
# define training configuration
# for your own use cases, you might want to change the data paths, model path, run_dir, and other hyperparameters

config = """
# data
data:
  instruct_data: "/content/drive/MyDrive/Colab_Projects/mistral-finetune/data/ultrachat_chunk_train.jsonl"  # Fill
  data: ""  # Optionally fill with pretraining data
  eval_instruct_data: "/content/drive/MyDrive/Colab_Projects/mistral-finetune/data/ultrachat_chunk_eval.jsonl"  # Optionally fill

# model
model_id_or_path: "/content/drive/MyDrive/Colab_Projects/mistral-finetune/mistral_models"  # Change to downloaded path
lora:
  rank: 64

# optim
# tokens per training steps = batch_size x num_GPUs x seq_len
# we recommend sequence lentgh of 32768
# If you run into memory error, you can try reduce the sequence length
seq_len: 8192
batch_size: 1
num_microbatches: 8
max_steps: 100
optim:
  lr: 1.e-4
  weight_decay: 0.1
  pct_start: 0.05

# other
seed: 0
log_freq: 1
eval_freq: 100
no_eval: False
ckpt_freq: 100

save_adapters: True  # save only trained LoRA adapters. Set to `False` to merge LoRA adapter into the base model and save full fine-tuned model

run_dir: "/content/drive/MyDrive/Colab_Projects/mistral-finetune/test_ultra"  # Fill
"""

# save the same file locally into the example.yaml file
import yaml
with open('example.yaml', 'w') as file:
    yaml.dump(yaml.safe_load(config), file)


In [ ]:
# make sure the run_dir has not been created before
# only run this when you ran torchrun previously and created the /content/drive/MyDrive/Colab_Projects/mistral-finetune/test_ultra file
# ! rm -r /content/drive/MyDrive/Colab_Projects/mistral-finetune/test_ultra

In [ ]:
# start training

!torchrun --nproc-per-node 1 -m train example.yaml

2025-01-09 00:21:56.101069: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-09 00:21:56.120439: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-09 00:21:56.142671: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-09 00:21:56.149399: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-09 00:21:56.167225: I tensorflow/core/platform/cpu_feature_guar

## Inference

In [ ]:
!pip install mistral_inference

In [ ]:
!pip install mistral

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.0/717.0 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.7/58.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.2/363.2 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.1/151.1 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.3/201.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.6/159.6 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install mistral.common mistral.model

ERROR: Could not find a version that satisfies the requirement mistral.model (from versions: none)
ERROR: No matching distribution found for mistral.model


In [ ]:
!pip install mistral
!pip install mistral.common mistral.model

ERROR: Could not find a version that satisfies the requirement mistral.model (from versions: none)
ERROR: No matching distribution found for mistral.model


In [ ]:
!pip install --upgrade mistral_inference

In [ ]:
!git clone https://github.com/mistralai/mistral_inference.git
!cd mistral_inference && pip install .

Cloning into 'mistral_inference'...
fatal: could not read Username for 'https://github.com': No such device or address
/bin/bash: line 1: cd: mistral_inference: No such file or directory


In [ ]:
from mistral_inference.model import Transformer
from mistral_inference.generate import generate

from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest


tokenizer = MistralTokenizer.from_file("/content/mistral_models/tokenizer.model.v3")  # change to extracted tokenizer file
model = Transformer.from_folder("/content/mistral_models")  # change to extracted model dir
model.load_lora("/content/test_ultra/checkpoints/checkpoint_000100/consolidated/lora.safetensors")

completion_request = ChatCompletionRequest(messages=[UserMessage(content="Explain Machine Learning to me in a nutshell.")])

tokens = tokenizer.encode_chat_completion(completion_request).tokens

out_tokens, _ = generate([tokens], model, max_tokens=64, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])

print(result)

ImportError: cannot import name 'Transformer' from 'mistral_inference.model' (/usr/local/lib/python3.10/dist-packages/mistral_inference/model.py)

In [ ]:
# prompt: goodbye banner

# Define a goodbye banner function
def display_goodbye_banner():
    print("\n")
    print("**************************************************")
    print("*                                                *")
    print("*               Goodbye! Exiting...              *")
    print("*                                                *")
    print("**************************************************")
    print("\n")

# Call the function when you want to display the banner
display_goodbye_banner()



**************************************************
*                                                *
*               Goodbye! Exiting...              *
*                                                *
**************************************************


